In [13]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [14]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [15]:
subject = 'Roraima - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [16]:
data = pd.read_csv('../Inputs Atuais/2008_01_model_input_RR.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data = data.drop('Roraima - Produção de Cimento (t)', axis=1)
data

,Ano,Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - IDH,Roraima - IDH Longevidade,Roraima - IDH Educacao,Roraima - IDH Renda,Contratação Comercial (Bi R$),...,Taxa Selic (%),Roraima - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Roraima - Consumo de Cimento (t)
0,2008,6.555833e+06,342849.112404,12.235418,5.474033e+06,0.736454,0.752834,0.733615,0.722576,5.483335e+08,...,11.760983,8.015090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.009000
1,2009,6.611657e+06,345335.485288,12.228462,5.482306e+06,0.736503,0.754681,0.734851,0.720114,5.302764e+08,...,9.472585,8.046321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.064000
2,2010,6.787712e+06,348962.483930,12.215749,5.534487e+06,0.736471,0.756148,0.735694,0.718135,5.046425e+08,...,9.332485,8.009229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.408000
3,2011,6.947966e+06,347630.972013,12.206783,5.574553e+06,0.736360,0.757242,0.736139,0.716628,4.854622e+08,...,10.985689,7.996819,22.217634,1001.541199,1506.456126,27.425647,21.330889,71.595088,1.830447,103.009000
4,2012,7.096137e+06,347411.232273,12.186539,5.609510e+06,0.736089,0.757859,0.736249,0.715498,4.867661e+08,...,8.159014,8.651837,21.157249,1001.690342,1515.527356,27.608612,21.117524,70.061650,1.898255,100.066000
5,2013,7.239848e+06,349584.100654,12.182243,5.648192e+06,0.735929,0.758347,0.735436,0.715372,5.225848e+08,...,7.913561,9.551098,18.013769,1001.817868,1528.998642,27.762053,20.876766,68.552790,1.943125,121.914000
6,2014,7.349320e+06,344289.736627,12.154911,5.667961e+06,0.735757,0.758460,0.735472,0.714737,5.676413e+08,...,10.350442,10.328300,14.644773,1001.940536,1542.131372,27.986329,20.647922,66.896135,1.956684,120.417571
7,2015,7.439258e+06,340498.356490,12.130631,5.685185e+06,0.735070,0.758081,0.733678,0.714965,5.179392e+08,...,12.454398,10.723767,19.007683,1001.988306,1541.272845,28.048825,20.665808,66.738045,1.910770,113.282057
8,2016,7.513614e+06,340512.038701,12.120014,5.705074e+06,0.735168,0.766432,0.738349,0.702779,4.524144e+08,...,13.111908,10.764101,24.437747,1002.055065,1514.905960,27.959130,20.665948,67.018448,1.902067,105.481371
9,2017,7.560265e+06,340005.357917,12.115483,5.723617e+06,0.733637,0.765744,0.735380,0.701759,4.213689e+08,...,9.479812,10.533542,26.318142,1002.155784,1495.611624,27.867978,20.547545,66.903661,1.943183,99.723000


In [17]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - IDH,Roraima - IDH Longevidade,Roraima - IDH Educacao,Roraima - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Roraima - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,6.555833e+06,342849.112404,12.235418,5.474033e+06,0.736454,0.752834,0.733615,0.722576,5.483335e+08,2.546644e+09,...,2.868108,11.760983,8.015090,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6.611657e+06,345335.485288,12.228462,5.482306e+06,0.736503,0.754681,0.734851,0.720114,5.302764e+08,2.679945e+09,...,2.305885,9.472585,8.046321,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6.787712e+06,348962.483930,12.215749,5.534487e+06,0.736471,0.756148,0.735694,0.718135,5.046425e+08,2.891930e+09,...,1.958246,9.332485,8.009229,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6.947966e+06,347630.972013,12.206783,5.574553e+06,0.736360,0.757242,0.736139,0.716628,4.854622e+08,3.091988e+09,...,2.079088,10.985689,7.996819,22.217634,1001.541199,1506.456126,27.425647,21.330889,71.595088,1.830447
4,7.096137e+06,347411.232273,12.186539,5.609510e+06,0.736089,0.757859,0.736249,0.715498,4.867661e+08,3.189506e+09,...,2.183282,8.159014,8.651837,21.157249,1001.690342,1515.527356,27.608612,21.117524,70.061650,1.898255
5,7.239848e+06,349584.100654,12.182243,5.648192e+06,0.735929,0.758347,0.735436,0.715372,5.225848e+08,3.093872e+09,...,2.403445,7.913561,9.551098,18.013769,1001.817868,1528.998642,27.762053,20.876766,68.552790,1.943125
6,7.349320e+06,344289.736627,12.154911,5.667961e+06,0.735757,0.758460,0.735472,0.714737,5.676413e+08,2.779340e+09,...,2.748397,10.350442,10.328300,14.644773,1001.940536,1542.131372,27.986329,20.647922,66.896135,1.956684
7,7.439258e+06,340498.356490,12.130631,5.685185e+06,0.735070,0.758081,0.733678,0.714965,5.179392e+08,2.561731e+09,...,2.501529,12.454398,10.723767,19.007683,1001.988306,1541.272845,28.048825,20.665808,66.738045,1.910770
8,7.513614e+06,340512.038701,12.120014,5.705074e+06,0.735168,0.766432,0.738349,0.702779,4.524144e+08,2.516859e+09,...,2.073464,13.111908,10.764101,24.437747,1002.055065,1514.905960,27.959130,20.665948,67.018448,1.902067
9,7.560265e+06,340005.357917,12.115483,5.723617e+06,0.733637,0.765744,0.735380,0.701759,4.213689e+08,2.497710e+09,...,2.177386,9.479812,10.533542,26.318142,1002.155784,1495.611624,27.867978,20.547545,66.903661,1.943183


In [18]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      63.064000
1      86.408000
2     103.009000
3     100.066000
4     121.914000
5     120.417571
6     113.282057
7     105.481371
8      99.723000
9      83.012000
10    121.120000
11    151.571000
12    144.989000
13    132.217000
14           NaN
Name: Roraima - Consumo de Cimento (t), dtype: float64

In [19]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input = train_input.fillna(0.0)
train_input

,Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - IDH,Roraima - IDH Longevidade,Roraima - IDH Educacao,Roraima - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Roraima - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.744850,-0.315007,1.401150,-1.746144,0.761548,-1.560951,-0.502066,1.373638,1.204752,-0.990234,...,2.026352,0.984445,-1.243205,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-1.589344,0.383838,1.268386,-1.652750,0.786997,-1.123635,0.058230,1.051274,0.820657,-0.546688,...,0.085558,-0.051710,-1.216598,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,-1.098918,1.403279,1.025732,-1.063622,0.770203,-0.776474,0.440255,0.792039,0.275392,0.158672,...,-1.114490,-0.115145,-1.248198,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,-0.652507,1.029031,0.854618,-0.611278,0.712391,-0.517408,0.641649,0.594618,-0.132596,0.824346,...,-0.697344,0.633403,-1.258770,0.026787,-1.750903,0.264474,-1.694225,2.033190,1.942658,-2.299835
4,-0.239756,0.967268,0.468245,-0.216621,0.570378,-0.371368,0.691446,0.446582,-0.104860,1.148829,...,-0.337667,-0.646476,-0.700745,-0.229627,-1.027431,0.358296,-1.009920,1.276809,1.179290,-0.506518
5,0.160572,1.577997,0.386241,0.220096,0.487127,-0.255786,0.323198,0.430184,0.657046,0.830615,...,0.422337,-0.757614,0.065357,-0.989758,-0.408818,0.497626,-0.436039,0.423317,0.428157,0.680157
6,0.465520,0.089910,-0.135410,0.443290,0.396938,-0.229205,0.339660,0.346932,1.615453,-0.215960,...,1.613108,0.345772,0.727473,-1.804421,0.186229,0.633454,0.402771,-0.387939,-0.396551,1.038753
7,0.716056,-0.975734,-0.598807,0.637745,0.037917,-0.318769,-0.473299,0.376803,0.558230,-0.940033,...,0.760923,1.298413,1.064381,-0.749418,0.417955,0.624574,0.636511,-0.324530,-0.475251,-0.175540
8,0.923184,-0.971888,-0.801440,0.862290,0.089103,1.657894,1.643124,-1.219209,-0.835563,-1.089341,...,-0.716758,1.596124,1.098742,0.563636,0.741797,0.351869,0.301046,-0.324036,-0.335662,-0.405692
9,1.053138,-1.114301,-0.887917,1.071645,-0.710639,1.495023,0.298008,-1.352812,-1.495939,-1.153056,...,-0.358019,-0.048438,0.902324,1.018338,1.230371,0.152313,-0.039871,-0.743775,-0.392804,0.681701


In [20]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0      63.064000
1      86.408000
2     103.009000
3     100.066000
4     121.914000
5     120.417571
6     113.282057
7     105.481371
8      99.723000
9      83.012000
10    121.120000
11    151.571000
Name: Roraima - Consumo de Cimento (t), dtype: float64

In [21]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - IDH,Roraima - IDH Longevidade,Roraima - IDH Educacao,Roraima - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Roraima - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.646085,-0.836784,-1.404774,0.642293,-2.414957,0.237678,-2.61321,-1.325075,1.108405,2.635823,...,-1.482779,-2.260447,0.345514,-1.395304,-1.927921,-2.672061,2.471949,-1.925114,-2.328206,2.648601


In [22]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    144.989
Name: Roraima - Consumo de Cimento (t), dtype: float64

In [23]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [24]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [26]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[2578411761, 2300846948, 927274891, 1733204137, 1043927829, 760656726, 2263415074, 3973429939, 1091699345, 1981399841, 2381054731, 2608001149, 2705443267, 2802588957, 3964388680, 707188862, 2746011932, 3327684883, 1044133524, 2136905600, 2978997730, 3506359930, 754533533, 3687114014, 324478596, 1672563330, 242528140, 850340384, 543317824, 3582453517, 3618250983, 1612420575, 2383690414, 4061273735, 3312656762, 2787611075, 404249340, 672711597, 2389252013, 2805520957, 1330091458, 2248897478, 1163420666, 3300745166, 3870060602, 3961565474, 3633110577, 631744562, 2384669624, 1257223996, 3472546706, 3956252553, 2915929258, 3317174501, 2951101840, 2249007136, 54061981, 2036930451, 3838412090, 2873969940, 1056602709, 1341474341, 696980027, 3450381686, 168193533, 1064199428, 3918688743, 2725774025, 22272458, 1325325899, 1853261323, 163272554, 2572358635, 137940809, 1678133715, 1917047823, 3885813653, 3809260104, 199924024, 1132865244, 1557720042, 1261260974, 3893247605, 959532377, 3123937858, 

2023-09-15 19:42:40.587852: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-09-15 19:42:41.126444: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-09-15 19:42:41.145812: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-09-15 19:42:41.169723: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-09-15 19:42:41.188886: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 932.4195556640625
winner_seed: 2300846948


Step: 2 ___________________________________________
val_loss: 1489.413330078125


Step: 3 ___________________________________________
val_loss: 1488.14453125


Step: 4 ___________________________________________
val_loss: 0.0018818883690983057
winner_seed: 1043927829


Step: 5 ___________________________________________
val_loss: 1434.0677490234375


Step: 6 ___________________________________________
val_loss: 32.90703582763672


Step: 7 ___________________________________________
val_loss: 2.3283064365386963e-10
winner_seed: 3973429939


Step: 8 ___________________________________________
val_loss: 1452.4326171875


Step: 9 ___________________________________________
val_loss: 1458.1470947265625


Step: 10 ___________________________________________
val_loss: 9.594717994332314e-06


Step: 11 ___________________________________________
val_loss: 1548.969482421875


Step: 12 ___________________________________________
val_loss: 903.

val_loss: 1484.2081298828125


Step: 99 ___________________________________________
val_loss: 208.38941955566406


final_seed: 3973429939


In [27]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 477ms/step - loss: 11674.3682 - val_loss: 21879.3242
Epoch 2/10000
1/1 [==============================] - 0s 15ms/step - loss: 10500.7021 - val_loss: 38436.2734
Epoch 3/10000
1/1 [==============================] - 0s 15ms/step - loss: 11433.0664 - val_loss: 17234.1133
Epoch 4/10000
1/1 [==============================] - 0s 15ms/step - loss: 11089.6719 - val_loss: 20443.0215
Epoch 5/10000
1/1 [==============================] - 0s 15ms/step - loss: 10770.3867 - val_loss: 19025.2207
Epoch 6/10000
1/1 [==============================] - 0s 15ms/step - loss: 10573.1445 - val_loss: 19017.1582
Epoch 7/10000
1/1 [==============================] - 0s 15ms/step - loss: 10346.9697 - val_loss: 18592.7480
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 10184.1045 - val_loss: 18271.7402
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 9985.6416 - val_loss: 18404.3945
Epoch 10/10000
1/1 [========

1/1 [==============================] - 0s 14ms/step - loss: 2862.8665 - val_loss: 7754.1572
Epoch 78/10000
1/1 [==============================] - 0s 14ms/step - loss: 2814.4080 - val_loss: 7668.4111
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 2766.9414 - val_loss: 7583.9780
Epoch 80/10000
1/1 [==============================] - 0s 14ms/step - loss: 2720.3633 - val_loss: 7500.8164
Epoch 81/10000
1/1 [==============================] - 0s 14ms/step - loss: 2674.7532 - val_loss: 7418.8838
Epoch 82/10000
1/1 [==============================] - 0s 14ms/step - loss: 2630.0310 - val_loss: 7338.2046
Epoch 83/10000
1/1 [==============================] - 0s 14ms/step - loss: 2586.2100 - val_loss: 7258.7949
Epoch 84/10000
1/1 [==============================] - 0s 14ms/step - loss: 2543.2520 - val_loss: 7180.6177
Epoch 85/10000
1/1 [==============================] - 0s 15ms/step - loss: 2501.0979 - val_loss: 7103.3589
Epoch 86/10000
1/1 [==============================] 

1/1 [==============================] - 0s 14ms/step - loss: 962.1843 - val_loss: 3855.2065
Epoch 154/10000
1/1 [==============================] - 0s 14ms/step - loss: 951.5490 - val_loss: 3826.8352
Epoch 155/10000
1/1 [==============================] - 0s 14ms/step - loss: 941.1221 - val_loss: 3798.8604
Epoch 156/10000
1/1 [==============================] - 0s 14ms/step - loss: 930.9072 - val_loss: 3771.2690
Epoch 157/10000
1/1 [==============================] - 0s 14ms/step - loss: 920.9001 - val_loss: 3744.0054
Epoch 158/10000
1/1 [==============================] - 0s 14ms/step - loss: 911.0842 - val_loss: 3717.1772
Epoch 159/10000
1/1 [==============================] - 0s 14ms/step - loss: 901.4725 - val_loss: 3690.6450
Epoch 160/10000
1/1 [==============================] - 0s 14ms/step - loss: 892.0466 - val_loss: 3664.5393
Epoch 161/10000
1/1 [==============================] - 0s 14ms/step - loss: 882.8059 - val_loss: 3638.7334
Epoch 162/10000
1/1 [==============================] 

1/1 [==============================] - 0s 14ms/step - loss: 543.9002 - val_loss: 2487.1069
Epoch 230/10000
1/1 [==============================] - 0s 14ms/step - loss: 541.5936 - val_loss: 2476.4897
Epoch 231/10000
1/1 [==============================] - 0s 14ms/step - loss: 539.3264 - val_loss: 2465.9758
Epoch 232/10000
1/1 [==============================] - 0s 14ms/step - loss: 537.1118 - val_loss: 2455.6194
Epoch 233/10000
1/1 [==============================] - 0s 14ms/step - loss: 534.9346 - val_loss: 2445.3596
Epoch 234/10000
1/1 [==============================] - 0s 14ms/step - loss: 532.8056 - val_loss: 2435.2468
Epoch 235/10000
1/1 [==============================] - 0s 14ms/step - loss: 530.7155 - val_loss: 2425.2483
Epoch 236/10000
1/1 [==============================] - 0s 14ms/step - loss: 528.6682 - val_loss: 2415.3643
Epoch 237/10000
1/1 [==============================] - 0s 14ms/step - loss: 526.6683 - val_loss: 2405.5959
Epoch 238/10000
1/1 [==============================] 

1/1 [==============================] - 0s 14ms/step - loss: 403.1319 - val_loss: 1718.2401
Epoch 306/10000
1/1 [==============================] - 0s 14ms/step - loss: 378.0030 - val_loss: 1579.4608
Epoch 307/10000
1/1 [==============================] - 0s 14ms/step - loss: 341.9849 - val_loss: 1495.6060
Epoch 308/10000
1/1 [==============================] - 0s 15ms/step - loss: 291.0832 - val_loss: 757.9769
Epoch 309/10000
1/1 [==============================] - 0s 14ms/step - loss: 170.5885 - val_loss: 700.9567
Epoch 310/10000
1/1 [==============================] - 0s 14ms/step - loss: 142.9972 - val_loss: 471.4768
Epoch 311/10000
1/1 [==============================] - 0s 14ms/step - loss: 242.9062 - val_loss: 785.0967
Epoch 312/10000
1/1 [==============================] - 0s 14ms/step - loss: 165.9067 - val_loss: 375.1867
Epoch 313/10000
1/1 [==============================] - 0s 15ms/step - loss: 105.4306 - val_loss: 380.0887
Epoch 314/10000
1/1 [==============================] - 0s 1

...

1/1 [==============================] - 0s 14ms/step - loss: 1.5412 - val_loss: 0.0581
Epoch 702/10000
1/1 [==============================] - 0s 14ms/step - loss: 1.1305 - val_loss: 0.1180
Epoch 703/10000
1/1 [==============================] - 0s 13ms/step - loss: 1.0293 - val_loss: 0.1203
Epoch 704/10000
1/1 [==============================] - 0s 14ms/step - loss: 2.0970 - val_loss: 0.0056
Epoch 705/10000
1/1 [==============================] - 0s 14ms/step - loss: 2.2491 - val_loss: 0.1551
Epoch 706/10000
1/1 [==============================] - 0s 13ms/step - loss: 3.1450 - val_loss: 0.1316
Epoch 707/10000
1/1 [==============================] - 0s 14ms/step - loss: 1.1212 - val_loss: 0.0253
Epoch 708/10000
1/1 [==============================] - 0s 14ms/step - loss: 1.7189 - val_loss: 0.1574
Epoch 709/10000
1/1 [==============================] - 0s 14ms/step - loss: 0.9141 - val_loss: 0.0550
Epoch 710/10000
1/1 [==============================] - 0s 13ms/step - loss: 2.1878 - val_loss: 0.0

1/1 [==============================] - 0s 14ms/step - loss: 3.5564 - val_loss: 0.0727
Epoch 782/10000
1/1 [==============================] - 0s 14ms/step - loss: 15.1140 - val_loss: 0.4841
Epoch 783/10000
1/1 [==============================] - 0s 14ms/step - loss: 6.9395 - val_loss: 0.2023
Epoch 784/10000
1/1 [==============================] - 0s 14ms/step - loss: 1.1169 - val_loss: 0.0022
Epoch 785/10000
1/1 [==============================] - 0s 14ms/step - loss: 1.6345 - val_loss: 0.0071
Epoch 786/10000
1/1 [==============================] - 0s 15ms/step - loss: 1.2577 - val_loss: 0.0122
Epoch 787/10000
1/1 [==============================] - 0s 14ms/step - loss: 0.8244 - val_loss: 0.0063
Epoch 788/10000
1/1 [==============================] - 0s 14ms/step - loss: 0.9098 - val_loss: 0.0032
Epoch 789/10000
1/1 [==============================] - 0s 15ms/step - loss: 1.0803 - val_loss: 0.0458
Epoch 790/10000
1/1 [==============================] - 0s 15ms/step - loss: 0.8274 - val_loss: 1.

1/1 [==============================] - 0s 14ms/step - loss: 0.9022 - val_loss: 1.7421e-04
Epoch 861/10000
1/1 [==============================] - 0s 14ms/step - loss: 0.7443 - val_loss: 1.8647e-05
Epoch 862/10000
1/1 [==============================] - 0s 14ms/step - loss: 0.7130 - val_loss: 0.0011
Epoch 863/10000
1/1 [==============================] - 0s 14ms/step - loss: 0.6760 - val_loss: 0.0041
Epoch 864/10000
1/1 [==============================] - 0s 14ms/step - loss: 0.6934 - val_loss: 1.0358e-04
Epoch 865/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.7086 - val_loss: 7.9514e-04
Epoch 866/10000
1/1 [==============================] - 0s 14ms/step - loss: 1.1194 - val_loss: 0.0238
Epoch 867/10000
1/1 [==============================] - 0s 14ms/step - loss: 0.7443 - val_loss: 4.7478e-04
Epoch 868/10000
1/1 [==============================] - 0s 14ms/step - loss: 0.6774 - val_loss: 0.0020
Epoch 869/10000
1/1 [==============================] - 0s 14ms/step - loss: 0.

1/1 [==============================] - 0s 14ms/step - loss: 0.6062 - val_loss: 2.6657e-04
Epoch 940/10000
1/1 [==============================] - 0s 14ms/step - loss: 0.7233 - val_loss: 3.5858e-04
Epoch 941/10000
1/1 [==============================] - 0s 14ms/step - loss: 0.6888 - val_loss: 8.5831e-04
Epoch 942/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.6440 - val_loss: 7.5102e-04
Epoch 943/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.7657 - val_loss: 6.8961e-04
Epoch 944/10000
1/1 [==============================] - 0s 14ms/step - loss: 0.6914 - val_loss: 0.0014
Epoch 945/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.5734 - val_loss: 3.2145e-04
Epoch 946/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.8441 - val_loss: 3.4371e-04
Epoch 947/10000
1/1 [==============================] - 0s 14ms/step - loss: 0.7086 - val_loss: 0.0024
Epoch 948/10000
1/1 [==============================] - 0s 14ms/step - 

1/1 [==============================] - 0s 14ms/step - loss: 0.6482 - val_loss: 0.0074
Epoch 1018/10000
1/1 [==============================] - 0s 14ms/step - loss: 0.6178 - val_loss: 5.0312e-06
Epoch 1019/10000
1/1 [==============================] - 0s 14ms/step - loss: 0.6360 - val_loss: 0.0027
Epoch 1020/10000
1/1 [==============================] - 0s 14ms/step - loss: 0.7664 - val_loss: 3.2036e-04
Epoch 1021/10000
1/1 [==============================] - 0s 14ms/step - loss: 0.5828 - val_loss: 0.0010
Epoch 1022/10000
1/1 [==============================] - 0s 14ms/step - loss: 0.8735 - val_loss: 0.0032
Epoch 1023/10000
1/1 [==============================] - 0s 15ms/step - loss: 0.7038 - val_loss: 9.2574e-04
Epoch 1024/10000
1/1 [==============================] - 0s 14ms/step - loss: 0.8272 - val_loss: 0.0127
Epoch 1025/10000
1/1 [==============================] - 0s 14ms/step - loss: 0.7439 - val_loss: 1.5314e-04
Epoch 1026/10000
1/1 [==============================] - 0s 14ms/step - los

In [28]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 49ms/step
[144.989](test_target) - [[151.57175]](prediction) = 6.582746826171871


In [31]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            train_input = train_input.fillna(0.0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            test_input = test_input.fillna(0.0)
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [32]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - IDH,Roraima - IDH Longevidade,Roraima - IDH Educacao,Roraima - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Roraima - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-0.970511,-1.141968,1.089794,-0.856005,-1.075992,-1.268873,-1.294281,1.266573,1.148205,-1.122043,...,1.308949,1.412346,-0.519434,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.405576,-0.151465,0.235656,-0.546903,1.332779,0.093630,0.153555,-0.088452,0.140893,-0.184474,...,-0.190804,-0.643250,1.398858,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.376087,1.293433,-1.325449,1.402908,-0.256787,1.175243,1.140727,-1.178121,-1.289098,1.306518,...,-1.118144,-0.769096,-0.879424,0.0,0.0,0.0,0.0,0.0,0.0,0.0


train_target:


,Roraima - Consumo de Cimento (t)
0,63.064
1,86.408
2,103.009


test_input:


...

,Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - IDH,Roraima - IDH Longevidade,Roraima - IDH Educacao,Roraima - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Roraima - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.646085,-0.836784,-1.404774,0.642293,-2.414957,0.237678,-2.61321,-1.325075,1.108405,2.635823,...,-1.482779,-2.260447,0.345514,-1.395304,-1.927921,-2.672061,2.471949,-1.925114,-2.328206,2.648601


test_target:


,Roraima - Consumo de Cimento (t)
12,144.989


1/1 [==============================] - 0s 39ms/step
Error: 6.582746826171871




[103.00894165039062,
 100.96183013916016,
 121.8829116821289,
 88.84410095214844,
 113.28642272949219,
 106.03822326660156,
 103.41442108154297,
 80.13499450683594,
 102.12574768066406,
 151.57174682617188]

In [33]:
display(targets)
display(predictions)

[100.066,
 121.914,
 120.41757142857142,
 113.28205714285713,
 105.48137142857142,
 99.723,
 83.012,
 121.12,
 151.571,
 144.989]

[103.00894165039062,
 100.96183013916016,
 121.8829116821289,
 88.84410095214844,
 113.28642272949219,
 106.03822326660156,
 103.41442108154297,
 80.13499450683594,
 102.12574768066406,
 151.57174682617188]

In [34]:
mae = mean_absolute_error(predictions, targets)
mae

18.133410824323384

In [35]:
porcentage = mae/np.mean(targets)
porcentage

0.1561104122702551